In all versions 2.x the narrative gets cleaned as before and products are cleaned twice. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time the same happens with the six most occuring ones. This run uses 20 different principle components.

__Initialing the data__

In [1]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_both.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 3


__Initializing the bag of words__

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [3]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=20)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(208872, 20)


In [4]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.18578615 -3.610759    0.13388786 ... -0.54652566  1.15203696
  -0.13296604]
 [ 3.99741841 -1.19752306  0.72257196 ... -1.05203991  0.83965275
  -0.35607695]
 [ 2.08671488  0.90268593  2.2992995  ... -0.38061559 -0.46367514
  -0.23677046]
 ...
 [ 8.09025722 -8.10508537 -1.75883809 ...  0.65896645 -3.15931774
  -2.70548853]
 [ 5.24479348 -5.0118416  -0.97544411 ... -0.18241553  1.40392981
  -0.29336061]
 [ 3.81963555 -2.1546713   0.58043305 ...  0.1914758   0.10798633
   0.57497495]]
                0          1         2         3         4         5  \
0        4.185786  -3.610759  0.133888  0.789195  0.120049 -0.061636   
1        3.997418  -1.197523  0.722572 -0.687759  0.304585  0.013855   
2        2.086715   0.902686  2.299300 -0.952044 -0.480680  0.329261   
3       24.306424 -15.933903  3.727168  8.876126 -9.842742  3.544956   
4        3.268770   0.855733  2.253802 -1.288721 -0.393339 -0.069052   
...           ...        ...       ...       ...       ...       ...   
2088

__One hot encoding of extra columns__

In [5]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(208872, 20)
(208872, 161)
(208872, 181)


__Split into train and test__

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(156654, 181)
(52218, 181)
(156654,)
(52218,)


__Application of linear SVM__

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3768
                                                 Checking or savings account       0.99      1.00      1.00      4764
                                                               Consumer Loan       0.93      0.79      0.85      2765
                                                                 Credit card       1.00      1.00      1.00      4680
                                                 Credit card or prepaid card       0.95      0.97      0.96      4653
                                                            Credit reporting       1.00      1.00      1.00      4776
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.86      0.92      6091
                                                       